# Fitting to the experimental data

In this notebook we will show how to load a CIF file, an experimental profile and how to perform a parameter fit.


#### Import Python packages

In [ ]:
import numpy as np

import easydiffraction as ed

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [ ]:
output_notebook()
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300

## Sample

#### Show a CIF file content

In [ ]:
cif_fname = 'PbSO4.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

#### Create a new job based on the CIF file

In [ ]:
job = ed.Job()
job.add_sample_from_file(cif_fname)

In [ ]:
phases = job.phases
phase = phases[0]

print(phases)
print(phase)

#### Visualise the structure

In [ ]:
structure = py3Dmol.view()
structure.addModel(phase.cif, 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

## Experiment

#### Show measured data as text

In [ ]:
meas_fname = 'D1A@ILL.xye'

with open(meas_fname, 'r') as f:
    content = f.read()
    
print('\n'.join(content.split('\n')[:11]))

#### Load the measured data

In [ ]:
meas_x, meas_y, meas_e = np.loadtxt(meas_fname, unpack=True)

#### Visualize the measured data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
show(fig)

## Analysis

#### The default job is of 1D powder neutron diffraction with constant wavelength type.

In [ ]:
print(f"Job type: {job.type}")

The data is of the same type, so we can load it directly into our job.

In [ ]:
job.add_experiment_from_file(meas_fname)

In [ ]:
print(f"Current calculator engine: {job.calculator}")

#### Generate the calculated data

Note that we do not need to pass the `x` values, since they are already known.

In [ ]:
calc_y_cryspy = job.calculate_profile()

#### Visualize both the measured and calculated data

In [ ]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Set scale manually

In [ ]:
job.phases[0].scale.enabled = True
job.phases[0].scale = 0.5

In [ ]:
calc_y_cryspy = job.calculate_profile(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Set wavelength manually

In [ ]:
job.sample.parameters.wavelength = 1.912

#### Set background points manually

In [ ]:
p1 = (meas_x[0], 200)
p2 = (meas_x[-1], 250)
points = [p1, p2]

job.set_background(points)

In [ ]:
calc_y_cryspy = job.calculate_profile(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
show(fig)

#### Define parameters to optimize

In [ ]:
job.pattern.scale.fixed = False
job.phases[0].scale.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.resolution_u.fixed = False
job.parameters.resolution_v.fixed = False
job.parameters.resolution_w.fixed = False
job.pattern.backgrounds[0][0].y.fixed = False
job.pattern.backgrounds[0][1].y.fixed = False

In [ ]:
print(job.pattern.scale)
print(job.phases[0].scale)
print(job.pattern.zero_shift)
print(job.parameters.resolution_u)
print(job.parameters.resolution_v)
print(job.parameters.resolution_w)
print(job.pattern.backgrounds[0][0].y)
print(job.pattern.backgrounds[0][1].y)

#### Perform the fit with Bumps Simplex method

In [ ]:
print(f"Available minimizers: {job.analysis.available_minimizers}")
print(f"Current minimizer: {job.analysis.current_minimizer}")
job.analysis.current_minimizer = 'Bumps_simplex'
print(f"Current minimizer: {job.analysis.current_minimizer}")

In [ ]:
job.fit()

In [ ]:
print("The fit has been successful: {}".format(job.fitting_results.success))
if job.fitting_results.success:    
    print("The gooodness of fit (chi2) is: {}".format(job.fitting_results.reduced_chi))
    print(job.pattern.scale)
    print(job.phases[0].scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.pattern.backgrounds[0][0])
    print(job.pattern.backgrounds[0][1])

In [ ]:
calc_y_cryspy = job.calculate_profile(meas_x)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(meas_x, meas_y, legend_label='Imeas', color='steelblue', line_width=2)
fig.line(meas_x, calc_y_cryspy, legend_label='Icalc (CrysPy)', color='orangered', line_width=2)
fig.line(meas_x, meas_y-calc_y_cryspy, legend_label='Imeas - Icalc (CrysPy)', color='olivedrab', line_width=2)
show(fig)